In [ ]:
# import packages
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, shape
import fiona
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

import plotly.express as px
from plotly.subplots import make_subplots

## Geodataframe

The CSO website was used to extract the Small Area and the LEA information. https://www.cso.ie/en/census/census2016reports/census2016smallareapopulationstatistics/

### Small Area
The Small Area information was extracted from:
https://data-osi.opendata.arcgis.com/datasets/small-areas-generalised-20m-osi-national-statistical-boundaries-2015/explore?location=53.407490%2C-8.379100%2C8.26



The file with .json was downloaded.


In [ ]:
# reading in the small area .geojson
small_area_gdf = gpd.read_file('Data/Shapefiles/small_area.geojson')

In [ ]:
small_area_gdf.head(1).transpose()

In [ ]:
small_area_gdf.columns = small_area_gdf.columns.str.lower()

small_area_gdf = small_area_gdf[['guid', 'countyname', 'edname','geometry'#,
#                                 'objectid', 'nuts1', 'nuts1name', 'nuts2', 
#                                  'nuts2name', 'nuts3', 'nuts3name', 'county', 
#                                  'csoed', 'osied', 'edname','sa_pub2011', 
#                                  'small_area', 'geogid', 'area', 'changecode', 
#                                  'esri_oid','shape__area', 'shape__length' 
                                ]]

small_area_gdf['countyname'] = small_area_gdf['countyname'].str.upper()
# renaming columns
small_area_gdf = small_area_gdf.rename(columns={"edname": "small_area_name",
                                               "countyname": "council"})

small_area_gdf['council'] = small_area_gdf['council'].str.replace('DÚN LAOGHAIRE-RATHDOWN', 'DUN LAOGHAIRE-RATHDOWN')

# dropping ' County' from values
small_area_gdf['council'] = small_area_gdf['council'] + ' COUNTY COUNCIL'

# county
small_area_gdf['council'] = small_area_gdf['council'].str.replace(' COUNTY COUNTY', ' COUNTY')
small_area_gdf['county'] = small_area_gdf['council'].str.replace(' COUNTY COUNCIL', '')
small_area_gdf['county'] = small_area_gdf['county'].str.replace(' CITY', '')

small_area_gdf['county'] = small_area_gdf['county'].str.replace('DUN LAOGHAIRE-RATHDOWN', 'DUBLIN')
small_area_gdf['county'] = small_area_gdf['county'].str.replace('FINGAL', 'DUBLIN')
small_area_gdf['county'] = small_area_gdf['county'].str.replace('SOUTH DUBLIN', 'DUBLIN')

small_area_gdf['county'] = small_area_gdf['county'].str.replace('LIMERICK AND', 'LIMERICK')
small_area_gdf['county'] = small_area_gdf['county'].str.replace('WATERFORD AND', 'WATERFORD')


small_area_gdf['small_area_name'] = small_area_gdf['small_area_name'].str.upper()

small_area_gdf = small_area_gdf[['guid','council',  'county', 'small_area_name', 'geometry']]

small_area_gdf['electoral_divisions'] = small_area_gdf['small_area_name'] +', CO.'+ small_area_gdf['county']

small_area_gdf.head()

In [ ]:
small_area_gdf['county'].unique()

In [ ]:
small_area_gdf.head(1).transpose()

#### Area of Small Area 

In [ ]:
area_small_area_gdf= small_area_gdf.to_crs({'init' :'epsg:2157'}) 

In [ ]:
for i in range(len(area_small_area_gdf)):
    area_small_area_gdf.loc[i,'area_in_m2'] = shape(area_small_area_gdf.loc[i,'geometry']).area
    
area_small_area_gdf['area_in_km2'] = area_small_area_gdf['area_in_m2'] /1000000

small_area_gdf['small_area_GUID_area_in_km2'] = area_small_area_gdf['area_in_km2']

In [ ]:
small_area_gdf.head(1).transpose()

## SAPS CSO Statistics

https://www.cso.ie/en/census/census2016reports/census2016smallareapopulationstatistics/

Download the full catalogue

Glossary is from there

In [ ]:
cso_glossary = pd.read_excel('Data\SAPS2016\SAPS_2016_Glossary.xlsx')
cso_glossary

In [ ]:
cso_glossary[['Column Names','Description of Field']]

In [ ]:
t6_datata = cso_glossary[['Column Names','Description of Field']][(cso_glossary['Column Names'].str.startswith('T6_')) &
                                                     (cso_glossary['Description of Field'].str.contains('households'))]

t6_datata['Description of Field'] = t6_datata['Description of Field'].str.replace(' \(No. of households\)', '')

t6_datata

In [ ]:
print(t6_datata['Column Names'].unique())

In [ ]:
t6_datata['Column Names'][t6_datata['Description of Field'].str.contains('Total')]


#### CSO Statistics

In [ ]:
# read the CSV file
small_area_cso_df = pd.read_csv('Data\SAPS2016\SAPS2016_SA2017.csv') 

# Will ensure that all columns are displayed
pd.set_option('display.max_columns', None) 

In [ ]:
# getting all columns that start with T6 (related to housig)
small_area_cso_df.columns[pd.Series(small_area_cso_df.columns).str.startswith('T6_')]

In [ ]:
small_area_cso_df = small_area_cso_df[['GUID', 

# TYPE OF ACCOMODATION
'T6_1_HB_H', 'T6_1_FA_H' ,'T6_1_BS_H', 'T6_1_CM_H','T6_1_NS_H', 'T6_1_TH', 

# YEAR BUILT                                 
'T6_2_PRE19H', 'T6_2_19_45H', 'T6_2_46_60H', 'T6_2_61_70H', 'T6_2_71_80H', 
'T6_2_81_90H', 'T6_2_91_00H','T6_2_01_10H', 'T6_2_11LH', 'T6_2_NSH', 'T6_2_TH', 

# TYPE OF OCCUPANCY
# 'T6_3_OMLH','T6_3_OOH', 'T6_3_RPLH', 'T6_3_RLAH' ,'T6_3_RVCHBH', 'T6_3_OFRH','T6_3_NSH', 'T6_3_TH',

# NUMBER OF ROOMS                                    
'T6_4_1RH', 'T6_4_2RH', 'T6_4_3RH','T6_4_4RH', 'T6_4_5RH', 'T6_4_6RH' ,
 'T6_4_7RH' ,'T6_4_GE8RH', 'T6_4_NSH', 'T6_4_TH']]

In [ ]:
small_area_cso_df.columns = ['guid', 
# TYPE OF ACCOMODATION
'House Bungalow', 'Flat Apartment', 'Bed-Sit', 'Caravan Mobile Home', 'Type Of Accommodation Not Stated', 'Total Type Of Accommodation', 
  
# YEAR BUILT  
'Pre 1919', '1919-1945', '1946-1960', '1961-1970', '1971-1980', '1981-1990',
'1991-2000', '2001-2010', '2011 Or Later', 'Year Built Not Stated','Total Year Built',

# TYPE OF OCCUPANCY
#        'Owned With Mortgage Or Loan', 'Owned Outright',
#        'Rented From Private Landlord ', 'Rented From Local Authority',
#        'Rented From Voluntary Co-Operative Housing Body',
#        'Occupied Free Of Rent', 'Type Of Occupancy Not Stated', 'Total Type Of Occupancy',

# NUMBER OF ROOMS  
'1 Room', '2 Rooms', '3 Rooms', '4 Rooms', '5 Rooms', '6 Rooms', '7 Rooms',
'8 Or More Rooms', 'Number Of Rooms Not Stated', 'Total Number of Rooms']

### Merging  Small Area with CSO

In [ ]:
#Merging Small Area with CSO data
small_area_gdf_merge = small_area_gdf.merge(small_area_cso_df, left_on='guid', right_on='guid', how='outer')

In [ ]:
small_area_gdf_merge.head(1).transpose()

### Percentages

In [ ]:
small_area_gdf_merge.columns = small_area_gdf_merge.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

In [ ]:
# TYPE OF ACCOMODATION
small_area_gdf_merge['house_bungalow_%'] = round(small_area_gdf_merge['house_bungalow']/small_area_gdf_merge['total_type_of_accommodation'],2)
small_area_gdf_merge['flat_apartment_%'] = round(small_area_gdf_merge['flat_apartment']/small_area_gdf_merge['total_type_of_accommodation'],2)
small_area_gdf_merge['bed_sit_%'] =round(small_area_gdf_merge['bed_sit']/small_area_gdf_merge['total_type_of_accommodation'],2)
small_area_gdf_merge['caravan_mobile_home_%']= round(small_area_gdf_merge['caravan_mobile_home']/small_area_gdf_merge['total_type_of_accommodation'],2)
small_area_gdf_merge['type_of_accommodation_not_statd_%'] = round(small_area_gdf_merge['type_of_accommodation_not_stated'] /small_area_gdf_merge['total_type_of_accommodation'] ,2)

# YEAR BUILT  
small_area_gdf_merge['pre_1919_%'] = round(small_area_gdf_merge['pre_1919']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['1919_1945_%'] = round(small_area_gdf_merge['1919_1945']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['1946_1960_%'] =round(small_area_gdf_merge['1946_1960']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['1961_1970_%']= round(small_area_gdf_merge['1961_1970']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['1971_1980_%'] = round(small_area_gdf_merge['1971_1980'] /small_area_gdf_merge['total_year_built'] ,2)
small_area_gdf_merge['1981_1990_%'] = round(small_area_gdf_merge['1981_1990']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['1991_2000_%'] = round(small_area_gdf_merge['1991_2000']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['2001_2010_%'] =round(small_area_gdf_merge['2001_2010']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['2011_or_later_%']= round(small_area_gdf_merge['2011_or_later']/small_area_gdf_merge['total_year_built'],2)
small_area_gdf_merge['year_built_not_stated_%'] = round(small_area_gdf_merge['type_of_accommodation_not_stated'] /small_area_gdf_merge['total_year_built'] ,2)

# NUMBER OF ROOMS 
small_area_gdf_merge['1_room_%'] = round(small_area_gdf_merge['1_room']/small_area_gdf_merge['total_number_of_rooms'],2)
small_area_gdf_merge['2_rooms_%'] = round(small_area_gdf_merge['2_rooms']/small_area_gdf_merge['total_number_of_rooms'],2)
small_area_gdf_merge['3_rooms_%'] =round(small_area_gdf_merge['3_rooms']/small_area_gdf_merge['total_number_of_rooms'],2)
small_area_gdf_merge['4_rooms_%']= round(small_area_gdf_merge['4_rooms']/small_area_gdf_merge['total_number_of_rooms'],2)
small_area_gdf_merge['5_rooms_%'] = round(small_area_gdf_merge['5_rooms'] /small_area_gdf_merge['total_number_of_rooms'] ,2)
small_area_gdf_merge['6_rooms_%'] = round(small_area_gdf_merge['6_rooms'] /small_area_gdf_merge['total_number_of_rooms'] ,2)
small_area_gdf_merge['7_rooms_%'] = round(small_area_gdf_merge['7_rooms'] /small_area_gdf_merge['total_number_of_rooms'] ,2)
small_area_gdf_merge['8_or_more_rooms_%'] = round(small_area_gdf_merge['8_or_more_rooms'] /small_area_gdf_merge['total_number_of_rooms'] ,2)
small_area_gdf_merge['number_of_rooms_not_stated_%'] = round(small_area_gdf_merge['number_of_rooms_not_stated'] /small_area_gdf_merge['total_number_of_rooms'] ,2)

In [ ]:
# getting all columns that start with T6 (related to housig)
small_area_gdf_merge.columns[pd.Series(small_area_gdf_merge.columns).str.endswith('_%')]

In [ ]:
small_area_gdf_merge.head(1).transpose()

In [ ]:
small_area_gdf_merge.columns

In [ ]:
# small_area_gdf_merge = small_area_gdf_merge[['guid', 'council', 'county', 'small_area_name', 'geometry','house_bungalow_%', 'flat_apartment_%', 'bed-sit_%',
#        'caravan_mobile_home_%', 'type_of_accommodation_not_statd_%',
#        'pre_1919_%', '1919_1945_%', '1946_1960_%', '1961_1970_%',
#        '1971_1980_%', '1981_1990_%', '1991_2000_%', '2001_2010_%',
#        '2011_or_later_%', 'year_built_not_stated_%', '1_room_%', '2_rooms_%',
#        '3_rooms_%', '4_rooms_%', '5_rooms_%', '6_rooms_%', '7_rooms_%',
#        '8_or_more_rooms_%', 'number_of_rooms_not_stated_%', 'small_area_guid_area_in_km2']]#



In [ ]:
small_area_gdf_merge['electoral_divisions'] = small_area_gdf_merge['small_area_name']+ ', CO.'+ small_area_gdf_merge['county']

In [ ]:
small_area_gdf_merge['electoral_divisions']

#### Vacant Dwelligs in 2022

In [ ]:
# read the CSV file
vacant_df = pd.read_csv('Data\SAPS2016\Vacant_Housing_Data_2022.csv') 

# Will ensure that all columns are displayed
pd.set_option('display.max_columns', None) 

# renaming columns
vacant_df = vacant_df.rename(columns={
                            "Statistic": "statistic" ,   
                            "CensusYear": "year" ,  
                            "Electoral Division": "electoral_divisions",
                            "UNIT": "unit",
                            "VALUE": "count"})

vacant_df['electoral_divisions'] = vacant_df['electoral_divisions'].str.upper()
vacant_df['statistic'] = vacant_df['statistic'].str.upper()

vacant_df['electoral_divisions'] = vacant_df['electoral_divisions'].str.replace('CO\.CO\.', 'CO.')


vacant_df[['electoral_divisions', 'county', 'ed_id']] = vacant_df['electoral_divisions'].str.split(', ', expand = True)

vacant_df['electoral_divisions'] = vacant_df['electoral_divisions'] + ', ' +vacant_df['county'] 

vacant_df =vacant_df[~(vacant_df['electoral_divisions'].isnull())] 

vacant_df['electoral_divisions'][vacant_df['electoral_divisions'].str.contains('GUALA MHÓR,, CO.MAYO')] ='GUALA MHÓR, CO.MAYO'
vacant_df['electoral_divisions'][vacant_df['electoral_divisions'].str.contains('AIRD MHÓR,, CO.WATERFORD')] ='AIRD MHÓR, CO.WATERFORD'
vacant_df['electoral_divisions'][vacant_df['electoral_divisions'].str.contains('AN LEARGAIDH MHÓR,, CO.DONEGAL')] ='AN LEARGAIDH MHÓR, CO.DONEGAL'
vacant_df['electoral_divisions'][vacant_df['electoral_divisions'].str.contains('CEANNÚIGH / MÁISTIR GAOITHE,.CO.KERRY')] ='CEANNÚIGH / MÁISTIR GAOITHE, CO.KERRY'

# pivoting table
vacant_df = pd.pivot_table(vacant_df, values='count', index= ['electoral_divisions'],
                    columns=['statistic']).reset_index()

vacant_df.columns = vacant_df.columns.str.lower().str.replace(' ', '_')
vacant_df = vacant_df.reset_index()

# selecting just relevant columns
vacant_df = vacant_df[['electoral_divisions', 'vacant_dwellings', 'housing_stock', 'vacancy_rate']]

# renaming columns
vacant_df = vacant_df.rename(columns={"vacancy_rate": "vacancy_rate_%"})

vacant_df['vacancy_rate_decimal'] = vacant_df['vacancy_rate_%'] / 100


vacant_df.head(5)

### Merging Vavant Dwelling to CSO

In [ ]:
#Merging Small Area with CSO data
small_area_gdf_merge = small_area_gdf_merge.merge(vacant_df, left_on='electoral_divisions', right_on='electoral_divisions', how='outer')

In [ ]:
small_area_gdf_merge.tail(50)

In [ ]:
blanks = small_area_gdf_merge[['electoral_divisions','electoral_divisions']][(small_area_gdf_merge['electoral_divisions'].isna()) |
                         (small_area_gdf_merge['electoral_divisions'].isna())].reset_index(drop = True)
blanks.head(50)

#### Save  Data

In [ ]:
small_area_gdf_merge.head(1).transpose()

In [ ]:
# small_area_gdf_merge.to_file('Data/Shapefiles/clean_small_area_gdf.geojson', index=False)

In [ ]:
small_area_gdf_merge.columns

#### Vacant Dwelling

In [ ]:
vacant_df = small_area_gdf_merge[['guid', 'council', 'county', 'small_area_name', 'geometry',
                                                           'vacant_dwellings', 'housing_stock','vacancy_rate_%', 'vacancy_rate_decimal']]


vacant_df.head() 

In [ ]:
vacant_df.to_csv('Data/clean_vacant_dwellings.csv', index=False)

In [ ]:
vacant_df.to_file('Data/Shapefiles/Small Area Sub-Data/clean_vacant_dwellings.geojson', index=False)

### Smaller parts of CSO and Melted

##### Accomodation

In [ ]:
small_area_cso_df_accomodation = small_area_gdf_merge[['guid', 'council', 'county', 'small_area_name', 'geometry',
'house_bungalow', 'flat_apartment', 'bed_sit', 'caravan_mobile_home', 
 'type_of_accommodation_not_stated', 'total_type_of_accommodation']]


small_area_cso_df_accomodation = pd.melt(small_area_cso_df_accomodation, id_vars=['guid', 'council', 'county', 'small_area_name', 'geometry'],
       value_vars = [ 'house_bungalow', 'flat_apartment', 'bed_sit',
       'caravan_mobile_home', 'type_of_accommodation_not_stated', 'total_type_of_accommodation'],
                                         var_name = 'accomodation', value_name='count')
small_area_cso_df_accomodation.head()

In [ ]:
small_area_cso_df_accomodation.to_file('Data\Shapefiles\Small Area Sub-Data\small_area_cso_df_accomodation.geojson', index=False)

##### Year Built

In [ ]:
small_area_cso_df_year_built = small_area_gdf_merge[['guid', 'council', 'county', 'small_area_name', 'geometry',
                                                     'pre_1919', '1919_1945', '1946_1960', '1961_1970', 
                                                     '1971_1980','1981_1990', '1991_2000', '2001_2010', 
                                                     '2011_or_later', 'year_built_not_stated', 'total_year_built' ]]

small_area_cso_df_year_built = pd.melt(small_area_cso_df_year_built, id_vars=['guid', 'council', 'county', 'small_area_name', 'geometry'],
                                    value_vars = ['pre_1919', '1919_1945', '1946_1960', '1961_1970', 
                                                 '1971_1980','1981_1990', '1991_2000', '2001_2010', 
                                                 '2011_or_later', 'year_built_not_stated','total_year_built'],
                                                  var_name = 'year_built', value_name='count')
                                         
small_area_cso_df_year_built.head()     

In [ ]:
small_area_cso_df_year_built.to_file('Data\Shapefiles\Small Area Sub-Data\small_area_cso_df_year_built.geojson', index=False)

##### Number of Rooms

In [ ]:
small_area_cso_df_number_of_rooms = small_area_gdf_merge[['guid', 'council', 'county', 'small_area_name', 'geometry',
                                    '1_room', '2_rooms','3_rooms', '4_rooms', '5_rooms', '6_rooms', '7_rooms',
                                    '8_or_more_rooms', 'number_of_rooms_not_stated','total_number_of_rooms' ]]


small_area_cso_df_number_of_rooms = pd.melt(small_area_cso_df_number_of_rooms, id_vars=['guid', 'council', 'county', 'small_area_name', 'geometry'],
       value_vars = [ '1_room', '2_rooms', '3_rooms', '4_rooms', '5_rooms', '6_rooms', '7_rooms',
       '8_or_more_rooms', 'number_of_rooms_not_stated', 'total_number_of_rooms'],
                                            var_name = 'number_of_rooms', value_name='count')

small_area_cso_df_number_of_rooms.head() 

In [ ]:
small_area_cso_df_number_of_rooms.to_file('Data\Shapefiles\Small Area Sub-Data\small_area_cso_df_number_of_rooms.geojson', index=False)